<a href="https://colab.research.google.com/github/Sarvveshp/NLP_Assignment_3/blob/main/NLP_ASS_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install spaCy and download the English model
!pip install -q spacy
!python -m spacy download en_core_web_sm

import spacy
import re
import pandas as pd

# Load the spaCy English model
nlp = spacy.load("en_core_web_sm")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 47.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
# Sample insurance statements
insurance_statements = [
    "On March 5, your deductible of $250 was processed for emergency consultation.",
    "Your deductible of $100 was applied on March 18 for physiotherapy treatment.",
    "April 2: Deductible paid: $300 for dental procedure.",
]

# Sample credit card transactions
credit_card_transactions = [
    {"date": "2024-03-05", "amount": 250.00, "description": "Hospital emergency service"},
    {"date": "2024-03-18", "amount": 100.00, "description": "Physiotherapy"},
    {"date": "2024-04-02", "amount": 300.00, "description": "Dental services"},
    {"date": "2024-04-10", "amount": 150.00, "description": "Groceries"},
]


In [3]:
def extract_deductibles(statements):
    data = []
    for statement in statements:
        doc = nlp(statement)
        amount = None
        date = None

        # Extract date using spaCy
        for ent in doc.ents:
            if ent.label_ == "DATE":
                date = ent.text

        # Extract dollar amount using regex
        match = re.search(r"\$\s?(\d+(\.\d{2})?)", statement)
        if match:
            amount = float(match.group(1))

        if date and amount:
            data.append({"statement": statement, "date": date, "amount": amount})

    return pd.DataFrame(data)


In [7]:
def match_deductibles_to_transactions(deductibles_df, transactions):
    matched = []
    for _, row in deductibles_df.iterrows():
        for txn in transactions:
            # Match month abbreviation from deductible date to transaction date
            try:
                txn_month = pd.to_datetime(txn["date"]).strftime("%B").lower()
                deductible_month = row["date"].split()[0].lower()
            except:
                continue

            if abs(txn["amount"] - row["amount"]) < 1e-2 and deductible_month in txn_month:
                matched.append({
                    "deductible_date": row["date"],
                    "amount": row["amount"],
                    "matched_txn_date": txn["date"],
                    "description": txn["description"]
                })

    # Return dataframe even if empty, with correct columns
    return pd.DataFrame(matched, columns=["deductible_date", "amount", "matched_txn_date", "description"])


In [8]:
def generate_monthly_summary(matched_df):
    matched_df['month'] = pd.to_datetime(matched_df['matched_txn_date']).dt.to_period('M')
    summary = matched_df.groupby('month')['amount'].sum().reset_index()
    summary.columns = ['Month', 'Total Deductibles']
    return summary


In [9]:
if matched_df.empty:
    print("No matches found. Please check the date or amount formats.")
else:
    summary_df = generate_monthly_summary(matched_df)
    print("Monthly Summary:")
    display(summary_df)


No matches found. Please check the date or amount formats.


In [10]:
# Extract deductibles
deductibles_df = extract_deductibles(insurance_statements)

# Match with transactions
matched_df = match_deductibles_to_transactions(deductibles_df, credit_card_transactions)

# Generate monthly summary
summary_df = generate_monthly_summary(matched_df)

# Display all results
print("Extracted Deductibles:")
display(deductibles_df)

print("Matched Transactions:")
display(matched_df)

print("Monthly Summary:")
display(summary_df)


Extracted Deductibles:


,statement,date,amount
0,"On March 5, your deductible of $250 was proces...",March 5,250.0
1,Your deductible of $100 was applied on March 1...,March 18,100.0
2,April 2: Deductible paid: $300 for dental proc...,April 2,300.0


Matched Transactions:


,deductible_date,amount,matched_txn_date,description,month
0,March 5,250.0,2024-03-05,Hospital emergency service,2024-03
1,March 18,100.0,2024-03-18,Physiotherapy,2024-03
2,April 2,300.0,2024-04-02,Dental services,2024-04


Monthly Summary:


,Month,Total Deductibles
0,2024-03,350.0
1,2024-04,300.0
